In [4]:
from lookup import LookupCreator
from sensepolar.polarity import WordPolarity
from sensepolar.bertEmbed import BERTWordEmbeddings
from sensepolar.polarDim import PolarDimensions

c:\Users\Alext\Anaconda3\envs\SensePOLAR\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# !pip install -U -q ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !pip install plotly
# !pip install chart-studio
# !pip install cufflinks

In [6]:
out_path = './antonyms/'

dimensions = [['cold.a.01', 'hot.a.01'], ['bad.a.01', 'good.a.01'], ['intelligent.a.01', 'unintelligent.a.01'], ['capable.a.01', 'incapable.a.01']]
    
lookupSpace = LookupCreator(dimensions, out_path)
lookupSpace.create_lookup_files()

antonym_path = out_path + "polar_dimensions.pkl"

model = BERTWordEmbeddings()

pdc = PolarDimensions(model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
pdc.create_polar_dimensions(out_path)


wp = WordPolarity(model, antonym_path=antonym_path, method='base-change', number_polar=4)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading antonyms from ./antonyms/lookup_anto_example_dict.pkl


In [7]:
word = "sun"
context = "The sun is shining today."
polarity_base_change = wp.analyze_word(word, context)
print("Word polarity using base change method: ", polarity_base_change)

Top: 1
Dimension: cold <------> hot
Definitions:  having a low or inadequate temperature or feeling a sensation of coldness or having been made cold by e.g. ice or refrigeration<------>used of physical heat; having a high or higher than desirable temperature or giving off heat or feeling or causing a sensation of heat or burning
Value:0.337848961353302


Top: 2
Dimension: intelligent <------> unintelligent
Definitions:  having the capacity for thought and reason especially to a high degree<------>lacking intelligence
Value: -0.124832034


Top: 3
Dimension: capable <------> incapable
Definitions:  (usually followed by `of') having capacity or ability<------>(followed by `of') lacking capacity or ability
Value: -0.10426792


Top: 4
Dimension: bad <------> good
Definitions:  having undesirable or negative qualities<------>having desirable or positive qualities especially those suitable for a thing specified
Value: -0.099258386


Word polarity using base change method:  [('cold', 'hot', 0.

In [8]:
import plotly.graph_objects as go

fig = go.Figure(data = go.Scatterpolar(r=[0.3378, 0.1248, 0.10427, 0.09925], theta=["hot", "intelligent", "capable", "bad"]))

fig.update_traces(fill="toself")
fig.update_layout(
    polar=dict(
    radialaxis_angle = -45,
    angularaxis = dict(
        direction ="clockwise",
        period= 4
        )
    )
)

fig.show()

In [9]:
wp = WordPolarity(model, antonym_path=antonym_path, method='projection', number_polar=4)
polarity_projection = wp.analyze_word(word, context)
print("Word polarity using projection method: ", polarity_projection)

Loading antonyms from ./antonyms/lookup_anto_example_dict.pkl
Top: 1
Dimension: cold <------> hot
Definitions:  having a low or inadequate temperature or feeling a sensation of coldness or having been made cold by e.g. ice or refrigeration<------>used of physical heat; having a high or higher than desirable temperature or giving off heat or feeling or causing a sensation of heat or burning
Value:3.5650296211242676


Top: 2
Dimension: intelligent <------> unintelligent
Definitions:  having the capacity for thought and reason especially to a high degree<------>lacking intelligence
Value: -2.1062632


Top: 3
Dimension: capable <------> incapable
Definitions:  (usually followed by `of') having capacity or ability<------>(followed by `of') lacking capacity or ability
Value: -1.3338804


Top: 4
Dimension: bad <------> good
Definitions:  having undesirable or negative qualities<------>having desirable or positive qualities especially those suitable for a thing specified
Value:0.29108220338821

In [10]:
import chart_studio as plotly
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import pandas as pd

init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


import os
from IPython.display import Image, display, HTML



ModuleNotFoundError: No module named 'chart_studio'

In [11]:
import plotly.graph_objects as go
import numpy as np
from collections import defaultdict


def plot_word_polarity(words, polar_dimension):
    # create dictionary with antonyms as key and (word,polar) as value
    antonym_dict = defaultdict(list)
    for w_i in range(len(words)):
        for antonym1, antonym2, value in polar_dimension[w_i]:
            antonym_dict[(antonym1, antonym2)].append((words[w_i], value))
    fig = go.Figure()

    # Add axes
    fig.add_shape(type="line", x0=-1, y0=0, x1=1, y1=0, line=dict(color='black', width=1))
    fig.add_shape(type="line", x0=-1, y0=0, x1=-1, y1=len(antonym_dict)/10, line=dict(color='black', width=1))
    fig.add_shape(type="line", x0=1, y0=0, x1=1, y1=len(antonym_dict)/10, line=dict(color='black', width=1))
    fig.add_shape(type="line", x0=0, y0=0, x1=0, y1=len(antonym_dict)/10, line=dict(color='black', width=1, dash='dash'))

    # Define color scale for the words
    colors = np.linspace(0, 1, len(words))

    # Add lines and markers for each word's polarity score
    counter = 0.01
    offset = 0.005
    for antonyms, polars in antonym_dict.items():
        for polar in polars:
            # Define color of the line and marker based on word's position in the list
            color = f'rgb({int(colors[words.index(polar[0])] * 255)}, {int((1-colors[words.index(polar[0])]) * 255)}, 0)'
            fig.add_shape(type="line", x0=polar[1], y0=counter, x1=0, y1=counter, 
                          line=dict(color=color, width=1))
            fig.add_trace(go.Scatter(x=[polar[1]], y=[counter], mode='markers', 
                                     marker=dict(color=color, symbol='square', size=10),
                          text=polar[0], showlegend=False))
            fig.add_annotation(x=polar[1] , y=counter, text=polar[0], font=dict(size=20), showarrow=False, xanchor='auto')
        fig.add_annotation(x=-1.1, y=counter, text=antonyms[0], font=dict(size=18), showarrow=False, xanchor='right')
        fig.add_annotation(x=1.1, y=counter, text=antonyms[1], font=dict(size=18), showarrow=False, xanchor='left')
        counter += offset + 0.1
    
    # Add legend with word and color
    legend_data = []
    for i, word in enumerate(words):
        color = f'rgb({int(colors[i] * 255)}, {int((1-colors[i]) * 255)}, 0)'
        legend_data.append({'x': 1.2, 'y': 0.9-i*0.03, 'text': word, 'showarrow': False,
                            'font': {'color': color, 'size': 12}})
    fig.update_layout(annotations=legend_data)

    # Set x and y axis titles
    fig.update_layout(
        xaxis_title=f"Polarity",
        yaxis_title="Words",
        xaxis_range=[-1, 1]
    )

    fig.show()
   


In [12]:
import plotly.graph_objects as go
import numpy as np
from collections import defaultdict


def plot_word_polarity(words, polar_dimension):
    # create dictionary with antonyms as key and (word,polar) as value
    antonym_dict = defaultdict(list)
    for w_i in range(len(words)):
        for antonym1, antonym2, value in polar_dimension[w_i]:
            antonym_dict[(antonym1, antonym2)].append((words[w_i], value))
    fig = go.Figure()

    print(antonym_dict)

    # Add axes
    fig.add_shape(type="line", x0=-1, y0=0, x1=1, y1=0, line=dict(color='black', width=1))
    fig.add_shape(type="line", x0=-1, y0=0, x1=-1, y1=len(antonym_dict)/10, line=dict(color='black', width=1))
    fig.add_shape(type="line", x0=1, y0=0, x1=1, y1=len(antonym_dict)/10, line=dict(color='black', width=1))
    fig.add_shape(type="line", x0=0, y0=0, x1=0, y1=len(antonym_dict)/10, line=dict(color='black', width=1, dash='dash'))

    # Define color scale for the words
    colors = np.linspace(0, 1, len(words))

    # Add lines and markers for each word's polarity score
    counter = 0.01
    offset = 0.005
    for i, (antonyms, polars) in enumerate(antonym_dict.items()):
        show_legend = True if i == 0 else False
        for polar in polars:
            # Define color of the line and marker based on word's position in the list
            color = f'rgb({int(colors[words.index(polar[0])] * 255)}, {int((1-colors[words.index(polar[0])]) * 255)}, 0)'
            fig.add_shape(type="line", x0=polar[1], y0=counter, x1=0, y1=counter, 
                          line=dict(color=color, width=1))
            fig.add_trace(go.Scatter(x=[polar[1]], y=[counter], mode='markers', 
                                     marker=dict(color=color, symbol='square', size=10),
                          name=polar[0], showlegend=show_legend))
            # fig.add_annotation(x=polar[1], y=counter, text=polar[0], font=dict(size=20), showarrow=True, xanchor='auto')
        fig.add_annotation(x=-1.1, y=counter, text=antonyms[0], font=dict(size=18), showarrow=False, xanchor='right')
        fig.add_annotation(x=1.1, y=counter, text=antonyms[1], font=dict(size=18), showarrow=False, xanchor='left')
        counter += offset + 0.1
    # Set x and y axis titles
    fig.update_layout(
        xaxis_title=f"Polarity",
        yaxis_title="Words",
        xaxis_range=[-1, 1],
        xaxis_autorange=True, yaxis_autorange=True
    )

    fig.show()


In [13]:
plot_word_polarity([word],[polarity_base_change])

defaultdict(<class 'list'>, {('cold', 'hot'): [('sun', 0.33784896)], ('intelligent', 'unintelligent'): [('sun', -0.124832034)], ('capable', 'incapable'): [('sun', -0.10426792)], ('bad', 'good'): [('sun', -0.099258386)]})


In [206]:
wp = WordPolarity(model, antonym_path, method="base-change", number_polar=4)

word = "sun"
context = "The sun is shining today."
polarity_base_change = wp.analyze_word(word, context)
    
word1 = "fire"
context1 = "the fire is burning"
polarity_base_change1 = wp.analyze_word(word1, context1)

word2 = "water"
context2 = "the water is super fresh"
polarity_base_change2 = wp.analyze_word(word2, context2)

word3 = "earth"
context3 = "the earth is beautiful"
polarity_base_change3 = wp.analyze_word(word2, context2)

word4 = "video"
context4 = "a video of a supernove"
polarity_base_change4 = wp.analyze_word(word2, context2)

print(polarity_base_change)


Loading antonyms from ./antonyms/lookup_anto_example_dict.pkl
Top: 1
Dimension: cold <------> hot
Definitions:  a mild viral infection involving the nose and respiratory passages (but not the lungs)<------>used of physical heat; having a high or higher than desirable temperature or giving off heat or feeling or causing a sensation of heat or burning
Value:0.13891442120075226


Top: 2
Dimension: intelligent <------> unintelligent
Definitions:  having the capacity for thought and reason especially to a high degree<------>lacking intelligence
Value: -0.10718196


Top: 3
Dimension: capable <------> incapable
Definitions:  (usually followed by `of') having capacity or ability<------>(followed by `of') lacking capacity or ability
Value: -0.09894575


Top: 4
Dimension: bad <------> good
Definitions:  that which is below standard or expectations as of ethics or decency<------>benefit
Value:0.08515263348817825


Top: 1
Dimension: cold <------> hot
Definitions:  a mild viral infection involving 

In [252]:
def plot_polar(word, polar_dimension):
    antonyms = []
    values = []
    for antonym1, antonym2, value in polar_dimension:
        values.append(abs(value))
        if value > 0:
            antonyms.append(antonym1)
        else:
            antonyms.append(antonym2)
    
    antonyms.append(antonyms[0])
    values.append(values[0])
    color = random.choice(px.colors.qualitative.Pastel)

    fig = go.Figure(
        go.Scatterpolar(
        name = word,
        r=values,
        theta=antonyms,
        line_color=color,
        fill="toself",

        )   
    )

    fig.update_layout(
        showlegend = True,
        height = 1000,
        polar=dict(
        radialaxis_angle = 45,
        angularaxis = dict(
            direction ="clockwise",
            period= len(antonyms)-1
            )
        )
    )

    fig.show()
    

In [240]:
import plotly.express as px
import random

print(px.colors.qualitative.Pastel)

['rgb(102, 197, 204)', 'rgb(246, 207, 113)', 'rgb(248, 156, 116)', 'rgb(220, 176, 242)', 'rgb(135, 197, 95)', 'rgb(158, 185, 243)', 'rgb(254, 136, 177)', 'rgb(201, 219, 116)', 'rgb(139, 224, 164)', 'rgb(180, 151, 231)', 'rgb(179, 179, 179)']


In [253]:
plot_polar(word, polarity_base_change)

In [134]:
from plotly.subplots import make_subplots
import math

def plot_word_polarity_polar_fig(words, polar_dimension):
    word_dict = {word: None for word in words}
    for idx, item in enumerate(polar_dimension):
        antonym_dict = {}
        for antonym1, antonym2, value in item:
            if value > 0:
                antonym_dict[antonym1] = value
            else:
                antonym_dict[antonym2] = abs(value)
        word_dict[words[idx]] = antonym_dict

    # print(word_dict)

    num_cols = 2
    num_rows = math.ceil(len(polar_dimension)/num_cols)
    specs = [[{"type":"polar"} for _ in range(num_cols)] for _ in range(num_rows)]

    fig = make_subplots(rows=num_rows, cols=num_cols, specs=specs)

    for idx, word in enumerate(word_dict):
        # print(word_dict[word].keys())
        fig.add_trace(
            go.Scatterpolar(
                r = list(word_dict[word].values()),
                theta = list(word_dict[word].keys()),
                name = word, 
            ), int(idx/2 + 1) , (idx%2 + 1) 
        )

    
    fig.update_traces(fill="toself")
    # fig.update_layout(
    #     polar=dict(
    #     radialaxis_angle = -45,
    #     angularaxis = dict(
    #         direction ="clockwise",
    #         period= len(polar_dimension[0])
    #         )
    #     )
    # )

    fig.show()

        

In [135]:
plot_word_polarity_polar_fig([word, word1],[polarity_base_change, polarity_base_change1])

In [76]:
plot_word_polarity_polar_fig([word, word1, word2, word3, word4],[polarity_base_change, polarity_base_change1, polarity_base_change2, polarity_base_change4, polarity_base_change4])

{'sun': {'cold': 0.33784896, 'unintelligent': 0.124832034, 'incapable': 0.10426792, 'good': 0.099258386}, 'fire': {'cold': 0.30669558, 'unintelligent': 0.089270495, 'good': 0.081530236, 'capable': 0.0321554}, 'water': {'incapable': 0.0804548, 'good': 0.06725921, 'unintelligent': 0.06658165, 'cold': 0.039990626}, 'earth': {'incapable': 0.0804548, 'good': 0.06725921, 'unintelligent': 0.06658165, 'cold': 0.039990626}, 'video': {'incapable': 0.0804548, 'good': 0.06725921, 'unintelligent': 0.06658165, 'cold': 0.039990626}}
1 1
1 2
2 1
2 2
3 1


In [14]:
plot_word_polarity_polar_fig([word],[polarity_base_change])

[[('cold', 'hot', 0.33784896), ('intelligent', 'unintelligent', -0.124832034), ('capable', 'incapable', -0.10426792), ('bad', 'good', -0.099258386)]]
('sun', 0.33784896)


ValueError: 
    Invalid value of type 'builtins.dict_values' received for the 'r' property of scatterpolar
        Received value: dict_values([[('sun', 0.33784896)], [('sun', -0.124832034)], [('sun', -0.10426792)], [('sun', -0.099258386)]])

    The 'r' property is an array that may be specified as a tuple,
    list, numpy array, or pandas Series

In [18]:
word1 = "fire"
context1 = "the fire is burning"
wp = WordPolarity(model, antonym_path=antonym_path, method='base-change', number_polar=4)
polarity_base_change1 = wp.analyze_word(word1, context1)

word = "sun"
context = "the sun is shining today and it is a beautiful hot day"
polarity_base_change = wp.analyze_word(word, context)

plot_word_polarity([word, word1],  [polarity_base_change, polarity_base_change1])

Loading antonyms from ./antonyms/lookup_anto_example_dict.pkl
Top: 1
Dimension: cold <------> hot
Definitions:  a mild viral infection involving the nose and respiratory passages (but not the lungs)<------>used of physical heat; having a high or higher than desirable temperature or giving off heat or feeling or causing a sensation of heat or burning
Value:0.1810009777545929


Top: 2
Dimension: bad <------> good
Definitions:  that which is below standard or expectations as of ethics or decency<------>benefit
Value:0.026403076946735382




IndexError: list index out of range

In [222]:
import matplotlib.pyplot as plt
from collections import defaultdict
from matplotlib.pyplot import cm
import numpy as np

def plot_word_polarity(words, polar_dimension):
    #create dictionary with antonyms as key and (word,polar) as value
    antonym_dict = defaultdict(list)
    for w_i in range(len(words)):
        for antonym1, antonym2, value in polar_dimension[w_i]:
            antonym_dict[(antonym1, antonym2)].append((words[w_i], value))
    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(10, 8))

    # Define colors and markers for each word
    colors = cm.rainbow(np.linspace(0, 1, len(words)))
    markers = ['$\u25AE$', '$\u25A1$', '$\u25CB$', '$\u25CF$', '$\u25B2$', '$\u25BC$']

    # Add horizontal and vertical lines
    ax.axhline(0, color='black', linewidth=1)
    ax.axvline(-10, color='black', linewidth=1)
    ax.axvline(10, color='black', linewidth=1)
    ax.axvline(0, color='black', linestyle='--')

    # Iterate through each antonym pair and its associated words
    counter = 0.1
    for antonyms, polars in antonym_dict.items():
        # Add antonym labels
        ax.annotate(antonyms[0], xy=(-10, counter), xytext=(-12, counter),
                    fontsize=15, ha='right', va='center', color='gray')
        ax.annotate(antonyms[1], xy=(10, counter), xytext=(12, counter),
                    fontsize=15, ha='left', va='center', color='gray')
        # Iterate through each word and its associated polarity
        for i, polar in enumerate(polars):
            # Add word labels
            ax.annotate(polar[0], xy=(polar[1], 0), xytext=(polar[1], counter+0.01),
                        fontsize=12, ha='center', va='bottom', color=colors[i])
            # Add data points
            ax.scatter(polar[1], counter, color=colors[i], s=200, marker=markers[i])
        # Add horizontal lines between antonym pairs
        ax.axhline(y=counter, color='gray', linestyle='--')
        counter += 0.2

    # Set axis limits and ticks
    ax.set_xlim([-10, 10])
    ax.set_xticks([-6, -4, 0, 4, 6])
    ax.set_xticklabels(['%d' % i for i in [-6, -4, 0, 4, 6]], fontsize=12, color='gray')
    ax.set_yticks([])
    ax.set_ylim([ 0, counter-0.1])

    # Remove spines and add title and labels
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_position('zero')
    ax.set_title("Word Polarity", fontsize=20)
    ax.set_xlabel("Polarity", fontsize=15, color='gray')
    ax.xaxis.set_label_coords(1.05, -0.01)
    ax.tick_params(axis='x', colors='gray')

    plt.tight_layout()
    plt.show()


In [277]:
# plot_word_polarity([word, word1],  [polarity_base_change, polarity_base_change1])

In [275]:
from collections import defaultdict
import plotly.graph_objects as go

def plot_word_polarity_2d(words, polar_dimension):
    # create dictionary with antonyms as key and (word,polar) as value
    antonym_dict = defaultdict(list)
    for w_i in range(len(words)):
        for antonym1, antonym2, value in polar_dimension[w_i]:
            antonym_dict[(antonym1, antonym2)].append((words[w_i], value))
    
    #create dictionary with word as key and (polar1,polar2) as value
    word_dict = defaultdict(list)
    for w_i in range(len(words)):
        for antonym1, antonym2, value in polar_dimension[w_i]:
            word_dict[words[w_i]].append(value)
    
    fig = go.Figure()

    antonyms = list(antonym_dict.keys())
    # Set axis titles
    fig.update_layout(
        xaxis_title=f"{antonyms[0][0]} - {antonyms[0][1]}",
        yaxis_title=f"{antonyms[1][0]} - {antonyms[1][1]}",
        xaxis_range=(-1, 1),
        yaxis_range=(-1, 1)
    )

    # Add lines for each antonym pair
    fig.add_shape(type="line", x0=-1, y0=0, x1=1, y1=0, line=dict(color='black', width=1))
    fig.add_shape(type="line", x0=0, y0=-1, x1=0, y1=1, line=dict(color='black', width=1))

    # Define color scale for the words
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']

    # Add markers for each word's polar value
    for word in word_dict:
        x, y = word_dict[word][:2]
        fig.add_trace(go.Scatter(x=[x], y=[y], mode='markers', marker=dict(color=colors.pop(), size=10), name=word))
    fig.show()


In [276]:
wp = WordPolarity(model, antonym_path=antonym_path, method='base-change', number_polar=2)
words = ["earth", "air"]
context = [ "the earth is shaking", "the air is blowing"]

polar_dimensions = []
for word, context in zip(words, context):
    polar_dimensions.append(wp.analyze_word(word, context))

plot_word_polarity_2d(words, polar_dimensions)

Loading antonyms from ./antonyms/lookup_anto_example_dict.pkl
Top: 1
Dimension: bad <------> good
Definitions:  having undesirable or negative qualities<------>having desirable or positive qualities especially those suitable for a thing specified
Value: -0.14473194


Top: 2
Dimension: intelligent <------> unintelligent
Definitions:  having the capacity for thought and reason especially to a high degree<------>lacking intelligence
Value: -0.11401199


Top: 1
Dimension: bad <------> good
Definitions:  having undesirable or negative qualities<------>having desirable or positive qualities especially those suitable for a thing specified
Value: -0.21808979


Top: 2
Dimension: intelligent <------> unintelligent
Definitions:  having the capacity for thought and reason especially to a high degree<------>lacking intelligence
Value: -0.1021716


